In [1]:
# Import modules 
import sys
from os import getcwd
from os.path import dirname
path = dirname(dirname(getcwd()))
sys.path.append(path)

In [2]:
from MintPy.interpret_toolkit import InterpretToolkit
from MintPy.utils import combine_top_features
import pandas as pd
import numpy as np
import pickle
from joblib import load

In [3]:
# Define target feature
TARGET_COLUMN = 'cat_rt'

# Load the model objects. In this case, we are using 3 
# popular models availabe in scikit-learn 
model_fname = ['RandomForest.pkl', 'LogisticRegression.pkl', 'GradientBoostingClassifier.pkl']
model_objs = [load(fname) for fname in model_fname]

# Load the training dataset 
data  = pd.read_csv('example_data.csv')
targets = data[TARGET_COLUMN].values

# only want to use these columns below
cols_to_use = ['dllwave_flux', 'dwpt2m', 'fric_vel', 'gflux', 'high_cloud',
            'lat_hf', 'low_cloud', 'mid_cloud', 'sat_irbt', 'sens_hf',
            'sfcT_hrs_ab_frez', 'sfcT_hrs_bl_frez', 'sfc_rough', 'sfc_temp',
            'swave_flux','temp2m', 'tmp2m_hrs_ab_frez', 'tmp2m_hrs_bl_frez',
            'tot_cloud', 'uplwav_flux','vbd_flux', 'vdd_flux','wind10m',
            'date_marker', 'urban','rural','d_ground','d_rad_d','d_rad_u',
            'hrrr_dT']

units = ['W m$^{-2}$', '$^\circ$C', 'm s$^{-1}$', 'W m$^{-2}$', '%', 'W m$^{-2}$', '%', '%', 
         '$^\circ$C', 'W m$^{-2}$', 'hrs', 'hrs', 'unitless','$^\circ$C', 'W m$^{-2}$', '$^\circ$C', 
         'hrs', 'hrs', '%', 'W m$^{-2}$', 'W m$^{-2}$', 'W m$^{-2}$', 'm s$^{-1}$', 'days', 'unitless', 
         'unitless', 'W m$^{-2}$', 'W m$^{-2}$', 'W m$^{-2}$', '$^\circ$C']

pretty_names = [r'$\lambda_{\downarrow}$', '$T_{d}$', '$V_{fric}$', 'Gflux', '$Cloud_{high}$',
 '$Lat_{F}$', '$Cloud_{low}$', '$Cloud_{mid}$', 'IRBT', '$Sens_{F}$',
 'Hours $T_{sfc}$ $> $0', 'Hours $T_{sfc}$ $<= $0', 'SfcRough', '$T_{sfc}$',
 '$I_{S}$', '$T_{2m}$', 'Hours $T_{2m}$ $> $0', 'Hours $T_{2m}$ $<= $0',
 '$Cloud_{Tot}$', r'$\lambda_{\uparrow}$', 'VBD', 'VDD', '10m wind',
 'Date marker', 'Urban', 'Rural', 'Diff1', 'Diff2', 'Diff3',
 '$T_{sfc}$ - $T_{2m}$']

feature_units = {c : u for c,u in zip(cols_to_use, units)}
readable_feature_names = {c : u for c,u in zip(cols_to_use, pretty_names)}

# get predictor subset of dataframe (only the predictors used in training the model)
examples = data[cols_to_use]

/anaconda3/envs/deep/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/anaconda3/envs/deep/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


# Initializing InterpretToolkit

To initialize `InterpretToolkit`, requires a model object (e.g., a trained sci-kit learn model object) or a list of model objects and examples and targets to evalute the model(s) on. `examples` and `targets` can be `pandas.DataFrame` or `numpy.array`, but if you are using arrays, then you must provide the feature names. 

In [4]:
myInterpreter = InterpretToolkit(model=model_objs, 
                             examples=examples, 
                             targets=targets,
                                )

# Permutation Importance

A first step in model interpretability is determining feature ranking. A popular model-agnostic method for determining predictor ranking is the permutation importance method. The permutation importance calculations in MintPy are performed by a stripped-down version of PermutationImportance (see https://permutationimportance.readthedocs.io/en/latest/ for additional details).  
 
Additional options for permutation importance include: 
* Performing bootstraping for confidence interval on predictor ranking (`nbootstrap`)
* Using multiple processors for parallelization (`njobs`)
* Subsample examples for speedier results (`subsample`)

We currently have 3 built-in error metrics for evaluating feature importance
* Area under the Curve (`'auc'`)
* Area under the Performance Diagram (`'aupdc'`)
* Brier Skill Score (`'bss'`)

In this example, we want the top 5 predictors 

In [5]:
results = myInterpreter.permutation_importance(
                                               n_vars=5, 
                                               evaluation_fn='auc', 
                                               nbootstrap=10, 
                                               subsample = 1.0,
                                               njobs=3
                                              )

Processing RandomForestClassifier...
Using 3 of processors to compute importance...
Starting on the important variable 1 out of 5...
Starting on the important variable 2 out of 5...
Starting on the important variable 3 out of 5...
Starting on the important variable 4 out of 5...
Starting on the important variable 5 out of 5...
Processing LogisticRegression...
Using 3 of processors to compute importance...
Starting on the important variable 1 out of 5...
Starting on the important variable 2 out of 5...
Starting on the important variable 3 out of 5...
Starting on the important variable 4 out of 5...
Starting on the important variable 5 out of 5...
Processing GradientBoostingClassifier...
Using 3 of processors to compute importance...
Starting on the important variable 1 out of 5...
Starting on the important variable 2 out of 5...
Starting on the important variable 3 out of 5...
Starting on the important variable 4 out of 5...
Starting on the important variable 5 out of 5...


In [6]:
# There is a built-in method to return a list of the top features for each model
important_vars = myInterpreter.get_important_vars(results, multipass=True)
print(important_vars)

# Since permutation importance can be a time-consuming for multiple predictors over a large dataset,
# MintPy also have a built-in function for saving the results 




# And a function for reloading the file and appropriately setting it as a class attribute 
# for the plotting function. 

important_vars = combine_top_features(important_vars, nvars=5)

{'RandomForestClassifier': ['dllwave_flux', 'dwpt2m', 'sfc_temp', 'temp2m', 'sfcT_hrs_ab_frez'], 'LogisticRegression': ['tmp2m_hrs_ab_frez', 'swave_flux', 'sfcT_hrs_ab_frez', 'sfcT_hrs_bl_frez', 'dwpt2m'], 'GradientBoostingClassifier': ['sfc_temp', 'dwpt2m', 'temp2m', 'high_cloud', 'tmp2m_hrs_bl_frez']}


In [7]:
fig = myInterpreter.plot_importance(multipass=True, 
                              metric = "Training AUC",
                              num_vars_to_plot=5)

# Saving the figures
myInterpreter.save_figure(fig=fig, 
                          fname='multipass_perm_importance.png', 
                          bbox_inches="tight", 
                          dpi=300, aformat="png")

# Fix the axis labelling for cases with only one panel!!!